# Forecasting of daily River Discharge (RD) based on temperature and previous precipitation levels

### Import of libraries

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import matplotlib.dates as mdates
from sklearn import linear_model,preprocessing
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error,r2_score
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.model_selection import train_test_split
import math

### LoadingData

In [41]:
def loadData(river,daysBefore,daysBefore2):
    data=pd.read_csv(river).values
    Xbefore = data[:,[0,2,3]]
    second_colum = Xbefore[:,2]
    second_colum2 = Xbefore[:,2]
    second_colum = np.roll(second_colum,daysBefore)
    second_colum2 = np.roll(second_colum2,daysBefore2)

    new_column = np.expand_dims(second_colum, axis=1)
    Xbefore = np.hstack((Xbefore, new_column))
    new_column = np.expand_dims(second_colum2, axis=1)
    Xbefore = np.hstack((Xbefore, new_column))
    split = daysBefore if daysBefore > daysBefore2 else daysBefore2
    
    X = Xbefore[split:,:]
    y = data[split:,1]
    

    return X,y


In [42]:
""" X, y = loadData('RD_data/RD_VougaR_pg.csv',2,5)
X_78 = [day for day in X[:,0] if '1980' in day]
print(len(X_78)) """

" X, y = loadData('RD_data/RD_VougaR_pg.csv',2,5)\nX_78 = [day for day in X[:,0] if '1980' in day]\nprint(len(X_78)) "

### Prep of Data

In [43]:
def apply_boxcox(y, lambda_val=0):
    y_float64 = np.array(y, dtype=np.float64)
    if lambda_val == 0:
        res = boxcox(y_float64)
    else:
        res = boxcox(y_float64, lambda_val)
    y = res[0]
    lambda_val = res[1]
    return y, lambda_val

In [44]:
def plot_learning_curve(X_trainN,y_train):

    # Define a range of alpha values to test
    alphas = [0.01,0.1,1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]
    ridge_cv = linear_model.RidgeCV(alphas=alphas, scoring=None)

    # Create learning curve
    train_sizes, train_errors, valid_errors = learning_curve(
        ridge_cv, X_trainN, y_train, train_sizes=np.linspace(0.1, 1.0, 10),scoring=lambda estimator, X, y: mean_squared_error(y, estimator.predict(X)))

    # Calculate mean and standard deviation of training and validation errors
    train_errors_mean = np.mean(train_errors, axis=1)
    train_errors_std = np.std(train_errors, axis=1)
    valid_errors_mean = np.mean(valid_errors, axis=1)
    valid_errors_std = np.std(valid_errors, axis=1)

    # Plot learning curve
    plt.figure()
    plt.title("Learning Curve for RidgeCV")
    plt.xlabel("Training Examples")
    plt.ylabel("Error")

    plt.grid()

    plt.plot(train_sizes, train_errors_mean, '-', color="r",
            label="Training score")
    plt.plot(train_sizes, valid_errors_mean, '-', color="g",
            label="Cross-validation score")
    plt.legend(loc="best")
    plt.show()

In [45]:
def preprocess_data(X, y, years):
    scaler = preprocessing.StandardScaler()
    
    yearsTraining= math.ceil((len(years)+1)/2) 
    X_trainN = []
    for i in range(0,yearsTraining):
        X_trainN += [row for row in X if years[i] in row[0]] 
    X_trainN = np.array(X_trainN)
    X_trainN = X_trainN[:,1:]
    y_train = y[:len(X_trainN)]
    X_testN = X[len(X_trainN):,1:]
    y_test = y[len(X_trainN):]

    #plot_learning_curve(X_trainN, y_train)

    X_train_scaled = scaler.fit_transform(X_trainN)
    y_train_boxcox, lambda_val = apply_boxcox(y_train)
    
    X_test_scaled = scaler.transform(X_testN)
    
    return X_train_scaled, y_train_boxcox, X_test_scaled, y_test, lambda_val


### Cost Function

In [46]:
def fit(X_train_scaled, y_train_boxcox, X_test_scaled, y_test, lambda_val):
    
    alphas = [0.01,0.1,1,2,3,4,5,6,7,8,9,10,100,1000]
    ridge_cv = linear_model.RidgeCV(alphas=alphas)

    ridge_cv.fit(X_train_scaled,y_train_boxcox)
    best_alpha = ridge_cv.alpha_
    y_predict = inv_boxcox(ridge_cv.predict(X_test_scaled), lambda_val)

    r2_training = ridge_cv.score(X_train_scaled, y_train_boxcox)
    rmse_training = np.sqrt(-1*ridge_cv.best_score_)

    r2 = r2_score(y_test, y_predict)
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))

    """ plt.plot(y_test, 'r', label='Original')
    plt.plot(y_predict, 'b', label='Predicted')
    ax = plt.gca()
    ax.set_ylim([0, ax.get_ylim()[1]*1.1])
    plt.legend()
    plt.show() """

    return r2, rmse, r2_training, rmse_training, best_alpha

### Test different memories

In [47]:
def test_for_river(river, years):

    best_r2 = (-np.inf,0,0)
    best_rmse = (np.inf,0,0)

    memory = [x for x in range(0,11)] + [x for x in range(30,360,30)]

    for i in memory:
        for j in memory:
            X, y = loadData(river, i, j)
            X_train_scaled, y_train_boxcox, X_test_scaled, y_test, lambda_val = preprocess_data(X, y, years)
            r2, rmse, _, _, _ = fit(X_train_scaled, y_train_boxcox, X_test_scaled, y_test, lambda_val)
            if r2 > best_r2[0]:
                best_r2 = (r2, i, j)
            if rmse < best_rmse[0]:
                best_rmse = (rmse, i, j)
            
    print("\n\n ----- TESTING FOR RIVER:", river + " ------ \n\n")
    print("Best i and j for best R2:", best_r2[1:])
    print("Best i and j for best RMSE:", best_rmse[1:])
    X, y = loadData(river, best_r2[1], best_r2[2])
    X_train_scaled, y_train_boxcox, X_test_scaled, y_test, lambda_val = preprocess_data(X, y, years)
    r2, rmse, r2_train, rmse_train, best_alpha = fit(X_train_scaled, y_train_boxcox, X_test_scaled, y_test, lambda_val)
    print("RMSE training:", rmse_train)
    print("R2 training:", r2_train)
    print("RMSE test:", rmse)
    print("R2 test:", r2)
    print("Best alpha:", best_alpha)

In [48]:
test_for_river("RD_data/RD_AntuaR_pg.csv", ["1985","1986","1987","1988","1989"])
test_for_river("RD_data/RD_MondegoR_pg.csv", ["1972","1973","1974","1975","1976"])
test_for_river("RD_data/RD_NeivaR_pg.csv", ["1984","1985","1986","1987","1988","1989"])
test_for_river("RD_data/RD_VougaR_pg.csv", ["1978", "1979", "1980"])



 ----- TESTING FOR RIVER: RD_data/RD_VougaR_pg.csv ------ 


Best i and j for best R2: (270, 4)
Best i and j for best RMSE: (330, 4)
RMSE training: 1.434446619456292
R2 training: 0.7267815790011438
RMSE test: 4.011815970372256
R2 test: 0.7984799860269483
Best alpha: 6.0
